**IMPORTS**

In [ ]:
# Import Functions
from importlib import reload
import utils
import anomaly_detection_functions

reload(utils)
reload(anomaly_detection_functions)
from utils import plot_loss_curve, plot_pca, plot_tsne, get_confusion_matrix, get_latent_representations_label, analyze_latent_variance, analyze_kl_divergence, linear_annealing, save_results_to_excel, save_trained_model
from anomaly_detection_functions import get_threshold_from_train, anomaly_detection, get_anomaly_detection_accuracy


import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, Model
import itertools
import pandas as pd
from datetime import datetime
import warnings
import keras
from tsfresh import extract_features
from tsfresh.feature_extraction.settings import EfficientFCParameters
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from scipy.stats import entropy
import scipy.stats
from tensorflow.keras.optimizers.schedules import CosineDecay
from importlib import reload
import os
#keras.saving.get_custom_objects().clear()

# Adjust pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # No wrapping, long rows won't be cut off
pd.set_option('display.max_colwidth', None)  # Show full column content (especially useful for long strings)

# Remove this after testing/debugging
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'  

AWS = False
s3 = None
BUCKET = ""


------------------------------------------------------------------

**PATH FILES**

In [ ]:
# Unprocessed Normal and Attack Data
preprocessed_normal_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Tw22206_L003_with_ecu_channel.csv" # Normal Unprocessed
preprocessed_DoS_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/dos_attack.csv" # Dos Unprocessed
preprocessed_Fuzzy_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/fuzzy_attack.csv" # Fuzzy Unprocessed
preprocessed_Replay_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/replay_attack.csv" # Replay Unprocessed - Test
preprocessed_Spoofing_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/spoofing_attack.csv" # Spoofing Unprocessed
preprocessed_Suspension_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/suspension_attack.csv" # Suspension Unprocessed - Hardest Attack Type

# Attack based on Channel
preprocessed_DoS_channel2_csv_path = r'/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/dos_attack_channel2.csv' # DoS on channel 2 (Red Channel)
preprocessed_Replay_channel2_csv_path = r'/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/replay_attack_channel2.csv' # Replay on channel 2 (Red Channel)
preprocessed_Suspension_channel2_csv_path = r'/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/suspension_attack_channel2.csv' # Suspension on channel 2 (Red Channel)
preprocessed_Spoofing_channel2_csv_path = r'/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/spoofing_attack_channel2.csv' # Spoofing on channel 2 (Red Channel)
preprocessed_new_Spoofing_channel2_csv_path = r'/Users/SCHUGD/Desktop/MasterThesis/Dataset/Attack_Logs/new_spoofing_attack_channel2.csv' # Spoofing on channel 2 (Red Channel)

# Unprocessed Channel Data
preprocessed_normal_channel0_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Channel_Logs/channel0Logs.csv"
preprocessed_normal_channel2_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Channel_Logs/channel2Logs.csv" # Red Channel
preprocessed_normal_channel4_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Channel_Logs/channel4Logs.csv" # Yellow Channel
preprocessed_normal_channel5_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Channel_Logs/channel5Logs.csv" # Green Channel

# Preprocessed Dataframe Data
processeddataframe_normal_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/train_dataframe.csv" # Normal CSV Dataframe (Turns Lists into Strings)
processeddataframe_DoS_csv_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/test_DoS_dataframe.csv" # DoS CSV Dataframe (Turns Lists into Strings)

# Preprocessed Pickle Data
processeddataframe_normal_pickle_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/train_Normal_dataframePickle.pkl" # Normal Pickle Dataframe
processeddataframe_DoS_pickle_path = r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/test_DoS_dataframePickle.pkl" # DoS Pickle Dataframe

# Current best model
best_model_path = "/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_LSTM_VAE_LD30_Beta25_NT800000_21-37.keras"


# PRELOAD Dataframe for Debug
DEBUG = False 

if DEBUG:
    column_names = ['timestamp' , 'arbitration_id' , 'channel' , 'dlc', 'data' , 'ecu']
    pre_dataframe = pd.read_csv(preprocessed_normal_csv_path, parse_dates=['timestamp'], header =0, names=column_names, nrows= 50000)



---------------------------------------------

**FEATURE SELECTION**

In [ ]:
def binary_encode(ids, num_bits):
    """
    Converts a list of CAN IDs (hexadecimal strings) to binary representation.
    
    Args:
        ids (list of str): List of CAN IDs as hexadecimal strings (e.g., ['0x101', '0x102']).
        num_bits (int): Number of bits to represent the IDs in binary format.
        
    Returns:
        numpy.ndarray: Array of binary representations, where each row is a binary vector.
    """
    binary_ids = []
    for id_str in ids:
        # Convert hexadecimal string to integer
        id_int = int(id_str, 16)
        # Convert integer to binary and pad with leading zeros
        binary_vector = [int(bit) for bit in f"{id_int:0{num_bits}b}"]
        binary_ids.append(binary_vector)
    return np.array(binary_ids)



def binary_encode_integers(ids, num_bits):
    """
    Converts a list of CAN IDs (integers) to binary representation.

    Args:
        ids (list of int): List of CAN IDs as integers (e.g., [452948266, 452946218]).
        num_bits (int): Number of bits to represent the IDs in binary format.

    Returns:
        numpy.ndarray: Array of binary representations, where each row is a binary vector.
    """
    binary_ids = []
    for id_int in ids:
        # Convert integer to binary and pad with leading zeros
        binary_vector = [int(bit) for bit in f"{id_int:0{num_bits}b}"]
        binary_ids.append(binary_vector)
    return np.array(binary_ids)

def compute_temporal_features_tsfresh(dataframe, custom_fc_parameters = None, ts_fresh = False):
    """
    Computes tsfresh features while preserving message-level granularity.
    
    Args:
        dataframe (pd.DataFrame): CAN data with 'timestamp', 'arbitration_id', 'data'.
        window_size (int): Rolling window size for local statistics.
    
    Returns:
        pd.DataFrame: Original dataframe with merged tsfresh features.
    """
    #dataframe = dataframe.copy()
    dataframe.sort_values(by="timestamp")
    # Check if timestamp is already a float and handle accordingly
    if dataframe["timestamp"].dtype != 'float64':
        # Convert the 'timestamp' column to datetime format and handle errors
        print("dates not floats")
        dataframe["timestamp"] = pd.to_datetime(dataframe["timestamp"], errors='coerce')

    # Check for NaT values and print those rows for inspection
    if dataframe["timestamp"].isna().sum() > 0:
        print(f"Found {dataframe['timestamp'].isna().sum()} NaT values in timestamp. Rows: {dataframe[dataframe['timestamp'].isna()]}")

    # Compute Inter-Arrival Time (IAT)
    dataframe["iat"] = dataframe.groupby("arbitration_id")["timestamp"].diff().fillna(0)

    # Check for NaN values in 'iat' and handle
    if dataframe["iat"].isna().sum() > 0:
        print("NaN values found in 'iat' column")
        dataframe["iat"] = dataframe["iat"].fillna(0)  # Replace NaNs with 0 or other logic

    # Compute Rolling Window Statistics per message
    window_size_seconds = 0.2 # interval between log 1 and 50 is approx 0.18seconds
    timestamps = dataframe["timestamp"].to_numpy()
    arbitration_ids = dataframe["arbitration_id"].to_numpy()

    # Initialize the result array
    msg_count_last_20ms = np.zeros(len(dataframe), dtype=int)

    # Define the window size in seconds (50 milliseconds = 0.050 seconds)

    # Iterate through each row and compute the count
    for i, (current_time, current_id) in enumerate(zip(timestamps, arbitration_ids)):
        mask = (timestamps >= current_time - window_size_seconds) & \
            (timestamps <= current_time) & \
            (arbitration_ids == current_id)
        msg_count_last_20ms[i] = np.sum(mask)

    # Add the result to the dataframe
    dataframe["msg_frequency"] = msg_count_last_20ms

    # Normalize msg_count_last_50ms to [0, 1]
    max_count = dataframe["msg_frequency"].max()
    if max_count > 0:
        dataframe["msg_frequency"] = dataframe["msg_frequency"] / max_count
    else:
        dataframe["msg_frequency"] = 0  # If max_count is 0, set all normalized values to 0

    dataframe["rolling_mean_iat"] = dataframe.groupby("arbitration_id")["iat"].transform(lambda x: x.rolling(20, min_periods=1).mean())
    dataframe["rolling_std_iat"] = dataframe.groupby("arbitration_id")["iat"].transform(lambda x: x.rolling(20, min_periods=1).std().fillna(0))

    # Check for NaN values in 'id' or 'time' columns and handle
    if dataframe['arbitration_id'].isna().sum() > 0 or dataframe['timestamp'].isna().sum() > 0:
        print("NaN values found in 'id' or 'time' columns")
        dataframe = dataframe.dropna(subset=['arbitration_id', 'timestamp'])

    if ts_fresh:
        # Prepare for tsfresh
        tsfresh_df = dataframe.rename(columns={"arbitration_id": "id", "timestamp": "time", "iat": "value"})
        tsfresh_df = tsfresh_df[['id', 'time', 'value']]


        if custom_fc_parameters is None:
            custom_fc_parameters = EfficientFCParameters()

        
        extracted_features = extract_features(tsfresh_df, column_id="id", column_sort="time", 
                                        default_fc_parameters=custom_fc_parameters)
        
        # Impute missing values
        impute(extracted_features)

        scaler = MinMaxScaler()
        normalized_features = scaler.fit_transform(extracted_features)
        normalized_extracted_features = pd.DataFrame(normalized_features, columns=extracted_features.columns, index=extracted_features.index)
        
        # Merge extracted features back to the original dataframe without losing time-series order
        dataframe = dataframe.merge(normalized_extracted_features, left_on="arbitration_id", right_index=True, how="left")
        dataframe.sort_values(by="timestamp")
    return dataframe



def calculate_entropy(row):
    byte_values = [row[f'data[{i}]'] for i in range(8)]
    # Compute frequency of each byte value (0-255)
    counts = np.bincount(byte_values, minlength=256)
    probabilities = counts / np.sum(counts)
    return entropy(probabilities, base=2)

def hamming_distance(payload1, payload2):
    """Calculate Hamming distance between two 8-byte payloads."""
    # Convert payloads to binary strings
    bin1 = ''.join(f"{int(byte, 16):08b}" for byte in payload1)
    bin2 = ''.join(f"{int(byte, 16):08b}" for byte in payload2)

    # Count differing bits
    return sum(b1 != b2 for b1, b2 in zip(bin1, bin2))

def compute_hamming_distances(dataframe, scaler, previous_x=1):
    """
    Computes the Hamming distance between each row's payload and the previous X messages.
    
    Args:
        dataframe (pd.DataFrame): CAN dataset with 'data' column.
        previous_x (int): Number of previous messages to compare with.

    Returns:
        pd.Series: A series containing the Hamming distances.
    """
    hamming_distances = []
    data_columns = [f'data[{i}]' for i in range(8)]  # Extracted byte columns

    for i in range(len(dataframe)):
        if i < previous_x:
            hamming_distances.append(0)  # No previous message to compare with
        else:
            prev_payload = dataframe.iloc[i - previous_x][data_columns].astype(str).tolist()
            curr_payload = dataframe.iloc[i][data_columns].astype(str).tolist()
            hamming_distances.append(hamming_distance(prev_payload, curr_payload))

    dataframe['hamming_distance'] = hamming_distances
    return dataframe

def feature_selection_preparation(file_name, phase ,pre_dataframe = None, rows = None, ts_fresh = False, ts_fresh_parameters = None, ts_fresh_custom_features = None):
    column_names_train = ['timestamp' , 'arbitration_id' , 'channel' , 'dlc', 'data' , 'ecu']
    column_names_test = ['timestamp', 'arbitration_id', 'dlc', 'data']
    if phase =='training':
        #dataframe = pd.read_csv(file_name, parse_dates=['timestamp'], header =0, names=column_names_train, nrows = rows)
        dataframe = pd.read_csv(file_name, header=0, names=column_names_train, nrows=rows, dtype={'timestamp': float})
    elif phase == 'debug':
        #print("check")
        dataframe = pre_dataframe  
    elif phase == 'test':
        #dataframe = pd.read_csv(file_name, parse_dates=['timestamp'], header =0, names=column_names_test + ['type'], nrows= rows)
        dataframe = pd.read_csv(file_name, header=0, names=column_names_test + ['type'], nrows=rows, dtype={'arbitration_id': int})
        # one hot encode type
        dataframe['type'] = dataframe['type'].apply(lambda x: 0 if x == 'R' else 1)
        # Print the count of 1s in the 'type' column
        count_of_ones = dataframe['type'].sum()
        print(f"Anomalies in 'type' column: {count_of_ones}")
    else:        
        print("invalid phase")
        return None
    scaler = MinMaxScaler()

    #print(dataframe.head(2))
    dataframe = compute_temporal_features_tsfresh(dataframe, custom_fc_parameters= ts_fresh_custom_features, ts_fresh= ts_fresh)
    #dataframe = compute_temporal_features(dataframe)
    
    dataframe = dataframe[dataframe['dlc'] == 8].reset_index(drop=True)

    # Extract data to indiviual columns
    data_columns = [f'data[{i}]' for i in range(8)]
    dataframe[data_columns] = dataframe['data'].str.split(' ', expand=True).iloc[:, :8]

    dataframe = compute_hamming_distances(dataframe, scaler,previous_x=5)  # Compare with the last 5 messages
    
    # Convert Data from Hexadecimal to Integers [0,255]
    for col in data_columns:
        dataframe[col] = dataframe[col].apply(lambda x: int(x, 16) if isinstance(x, str) else x)

    # Entropy feature
    dataframe['payload_entropy'] = dataframe.apply(calculate_entropy, axis=1)    

    num_bits = 29  # Standard for CAN IDs
    binary_encoded_ids = binary_encode_integers(dataframe['arbitration_id'].tolist(), num_bits)
    binary_encoded_df = pd.DataFrame(binary_encoded_ids, columns=[f'bit_{i}' for i in range(num_bits)])

    # Add binary-encoded IDs to the original DataFrame
    #dataframe = dataframe.drop(columns=['arbitration_id']).reset_index(drop=True)
    dataframe = pd.concat([binary_encoded_df, dataframe], axis=1)
    
    dataframe[data_columns] = scaler.fit_transform(dataframe[data_columns])
    dataframe['timestamp'] = scaler.fit_transform(dataframe[['timestamp']])
    dataframe['payload_entropy'] = scaler.fit_transform(dataframe[['payload_entropy']])
    dataframe['hamming_distance'] = scaler.fit_transform(dataframe[['hamming_distance']])


    # Create a combined feature column, ensuring everything is a float
    #dataframe['features'] = dataframe.apply(lambda row: np.concatenate([row[[f'bit_{i}' for i in range(num_bits)]].values , row[data_columns].values]), axis=1)
    if ts_fresh:
        dataframe['features'] = dataframe.apply(
            lambda row: np.concatenate([
                row[[f'bit_{i}' for i in range(num_bits)]].values, 
                row[data_columns].values, 
                row[ts_fresh_parameters].values
            ]), axis=1
        )
    else:
        dataframe['features'] = dataframe.apply(
        lambda row: np.concatenate(
            [row[[f'bit_{i}' for i in range(num_bits)]].values,  # Binary data columns
            row[data_columns].values,  # Other data columns
            np.array([row['msg_frequency'], row['timestamp']])  # Additional features
            ]), axis=1)
        
        # row['iat'], row['msg_frequency'],  row['rolling_mean_iat'] , row['rolling_std_iat'],
        #   np.array([ row['payload_entropy'],   
        #              row['hamming_distance']
        #              ])]), axis=1)
        #row['payload_entropy'], row['hamming_distance'], 
        
    nan_counts = dataframe.isna().sum()
    if nan_counts.any():
        print("NaN values found:\n", nan_counts[nan_counts > 0])
    print(np.any(np.isinf(dataframe.select_dtypes(include=[np.number]))))
    return dataframe


def create_sliding_windows(data, labels=None, window_size=5, stride=1, anomaly_window_ratio = 0.5):
    # Generates sliding windows for both features and labels.
    X = np.array([data[i:i+window_size] for i in range(0, len(data) - window_size + 1, stride)], dtype=np.float32)
    print("Original window that works: ", X.shape)  # (991, 50, 40)
    """
    # Create array with the same structure but with added features
    num_window_features = 3
    augmented_X = np.zeros((X.shape[0], X.shape[1], X.shape[2] + num_window_features), dtype=np.float32)

    # Copy all original data (already normalized, so we keep as is)
    augmented_X[:, :, :X.shape[2]] = X

    # Calculate window features and add to each data point in that window
    for i, window in enumerate(X):
        # Split window into meaningful components
        arbitration_ids = window[:, :29]  # First 29 bits (arbitration ID)
        payloads = window[:, 29:37]       # Next 8 bits (payload)
        entropy_values = window[:, 37]    # Next value (entropy)
        hamming_distances = window[:, 38] # Next value (hamming distance)
        timestamps = window[:, 39]        # Last value (timestamp)
    
        # Calculate window-level features
        unique_arbitration_ids = len(np.unique(arbitration_ids, axis=0))
        mean_entropy = np.mean(entropy_values)
        var_entropy = np.var(entropy_values)
        mean_hamming = np.mean(hamming_distances)
        inter_arrival_times = np.diff(timestamps)
        mean_inter_arrival = np.mean(inter_arrival_times) if len(inter_arrival_times) > 0 else 0
        
        # Store window features temporarily
        window_features = np.array([
            #unique_arbitration_ids,
            mean_entropy,
            #var_entropy,
            mean_hamming,
            mean_inter_arrival
        ])

        #print("mean entropy", mean_entropy)
        #print("mean IAT", mean_inter_arrival)

        # Add these window features to all data points in this window
        for j in range(num_window_features):
            augmented_X[i, :, X.shape[2] + j] = window_features[j]

    print("Shape after adding window features:", augmented_X.shape)
    X = augmented_X
    """
    
    if labels is not None:
        labels = labels.values

        # Initialize an empty list to store the labels
        y = []

        # Define threshold: At least 50% of the window should contain 1s
        threshold = anomaly_window_ratio * window_size

        for i in range(0, len(labels) - window_size + 1, stride):
            # Extract the current window of labels
            window = labels[i:i+window_size]
            # Check if there is at least one '1' in the window
            if np.sum(window == 1) >= threshold:  # Count 1s and compare to threshold
                y.append(1)
            else:
                y.append(0)  # If there's not enough 1s in window, mark this window as normal

        # Convert the list to a numpy array
        y = np.array(y, dtype=np.float32)
        count_of_ones = np.sum(y)
        print(f"Anomalies in 'y' array: {count_of_ones}")
        return X, y
    return X


if DEBUG:
    ids = ['0x101', '0x102', '0x103']
    num_bits = 11  # Standard for CAN IDs
    binary_encoded_ids = binary_encode(ids, num_bits)

    # Print the result
    print(binary_encoded_ids)

In [ ]:
def convert_to_tensorflow(featureframe, labels=None, batch_size=32, window_size=5, stride=1, split_ratio = 0.8, window_anomaly_ratio = 0.5):
    # Convert feature list to NumPy array
    input_data = np.array(featureframe.tolist(), dtype=np.float32)
    before_window_shape = input_data.shape

    # Check if Train or Test dataframe
    if labels is not None:
        input_data, labels = create_sliding_windows(input_data, labels, window_size, stride, window_anomaly_ratio)
        print("x before tensor", len(input_data))
        print("y before tensor", len(labels))
        # todo: check windows 
        labels = np.array(labels, dtype=np.float32)  # Ensure labels are NumPy arrays
        model_input = tf.data.Dataset.from_tensor_slices((input_data, labels))

        dataset_size = len(input_data)
        test_size = int(dataset_size*split_ratio)

        #Shuffle , decrease size of buffer for faster shuffling
        #buffer_size=min(50000, dataset_size)
        #model_input = model_input.shuffle(buffer_size=buffer_size, reshuffle_each_iteration=False, seed= SEED)

        # Split to train and anomaly threshold set
        train_dataset = model_input.take(test_size)
        val_dataset = model_input.skip(test_size)

        # Batch
        test_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        print(f"Feature shape BEFORE sliding window: {before_window_shape}")
        print(f"Feature shape AFTER sliding window: {input_data.shape}")
        return test_dataset, val_dataset  

    else:
        input_data = create_sliding_windows(input_data, labels=None, window_size=window_size, stride=stride)
        model_input = tf.data.Dataset.from_tensor_slices(input_data)

        # Get Size of training set
        dataset_size = len(input_data)
        train_size = int(dataset_size * split_ratio)
        
        #Shuffle , decrease size of buffer for faster shuffling
        #buffer_size=min(1000, dataset_size)
        #model_input = model_input.shuffle(buffer_size=buffer_size, reshuffle_each_iteration=True, seed= SEED)

        # Split to train and anomaly threshold set
        train_dataset = model_input.take(train_size)
        val_dataset = model_input.skip(train_size)

        # Batch
        train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        print(f"Feature shape BEFORE sliding window: {before_window_shape}")
        print(f"Feature shape AFTER sliding window: {input_data.shape}")
        return train_dataset, val_dataset  

    # Apply batching
    model_input = model_input.batch(batch_size)
    print(f"Feature shape BEFORE sliding window: {before_window_shape}")
    print(f"Feature shape AFTER sliding window: {input_data.shape}")
    print(f"Successfully prepared model input data.")
    return model_input

In [ ]:
def save_datasets(train_dataset, val_dataset, path=r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/new_Channel2_Full_43Features_WindowFeatures_1024Batch_50Window_5Stride.npz"):
    """
    Save TensorFlow datasets to disk, preserving their exact structure.
    Works with batched or unbatched datasets with complex shapes.
    
    Args:
        train_dataset: A TensorFlow dataset
        val_dataset: A TensorFlow dataset
        path: Path to save the file (will be converted to .npz)
    """
    # Ensure directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Save specs for reconstruction
    train_spec = tf.data.experimental.get_structure(train_dataset)
    val_spec = tf.data.experimental.get_structure(val_dataset)
    
    # Create temporary folders for TFRecords
    temp_train_path = os.path.join(os.path.dirname(path), "temp_train")
    temp_val_path = os.path.join(os.path.dirname(path), "temp_val")
    
    # Save datasets to TFRecord format
    tf.data.experimental.save(train_dataset, temp_train_path)
    tf.data.experimental.save(val_dataset, temp_val_path)
    
    # Save metadata (paths and specs) to the specified file
    np.savez_compressed(
        path.replace('.pkl', ''),
        train_path=np.array(temp_train_path),
        val_path=np.array(temp_val_path),
        train_spec_dtype=np.array(train_spec._dtype.as_numpy_dtype),  # Fix dtype saving
        val_spec_dtype=np.array(val_spec._dtype.as_numpy_dtype),  # Fix dtype saving
        train_spec_shape=np.array(train_spec._shape),
        val_spec_shape=np.array(val_spec._shape)
    )
    
    print(f"Datasets saved. Metadata at {path.replace('.pkl', '.npz')}")
    print(f"Train dataset at {temp_train_path}")
    print(f"Val dataset at {temp_val_path}")

def load_datasets(path=r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/Channel2_Full_43Features_WindowFeatures_1024Batch_50Window_5Stride.npz"):
    """
    Load TensorFlow datasets from disk with their original structure preserved.
    
    Args:
        path: Path where datasets were saved (will be converted to .npz)
        
    Returns:
        tuple: (train_dataset, val_dataset)
    """
    metadata = np.load(path.replace('.pkl', '.npz'), allow_pickle=True)
    # Extract paths from metadata (convert from numpy array to string)
    train_path = metadata['train_path'].item()  # Convert numpy array to string
    val_path = metadata['val_path'].item()

    train_spec = tf.TensorSpec(
        shape=tuple(metadata['train_spec_shape']),
        dtype=metadata['train_spec_dtype'].item()  # Convert back to NumPy dtype
    )

    val_spec = tf.TensorSpec(
        shape=tuple(metadata['val_spec_shape']),
        dtype=metadata['val_spec_dtype'].item()  # Convert back to NumPy dtype
    )
    
    # Load datasets
    train_dataset = tf.data.experimental.load(train_path, train_spec)
    val_dataset = tf.data.experimental.load(val_path, val_spec)
    
    return train_dataset, val_dataset



Tsfresh parameters

In [ ]:
custom_fc_parameters = {
    # Statistical Features
    "mean": None,
    "skewness": None,
    "kurtosis": None,
    
    # Temporal Dependency Features
    "autocorrelation": [{"lag": 1}],  # Detects repeated patterns (useful for replay attacks)
    "agg_autocorrelation": [{"f_agg": "mean", "maxlag": 5}],  # Aggregate autocorrelation over multiple lags
    "agg_linear_trend": [{"attr": "slope", "chunk_len": 5, "f_agg": "mean"}],  # Detects trends in timing
    
    # Frequency Features
    "fft_coefficient": [{"coeff": 1, "attr": "real"}],  # Extract 1st FFT coefficient (frequency analysis)
    "fft_aggregated": [{"aggtype": "centroid"}],  # Spectral centroid (detects abnormal frequency changes)

    # Entropy Features (Detects randomness or spoofing)
    "approximate_entropy": [{"m": 2, "r": 0.1}],  # Measures unpredictability (spoofed messages might increase entropy)
    "binned_entropy": [{"max_bins": 10}],  # Useful for detecting shifts in data distribution

    # Complexity & Energy-Based Features
    "cid_ce": [{"normalize": True}],  # Measures signal complexity (can detect spoofing)
    "absolute_sum_of_changes": None,  # Captures sudden payload/timing shifts (useful for injection attacks)
    "energy_ratio_by_chunks": [{"num_segments": 5, "segment_focus": 2}],  # Detects anomalies in energy distribution

    # Stationarity & Distribution-Based Features
    "augmented_dickey_fuller": [{"attr": "teststat"}],  # Tests stationarity (useful for detecting sudden changes)
    "benford_correlation": None,  # Checks if data follows natural distribution (detects synthetic spoofing)

    }

"""  
    ALL Features extracted ATM, add parameters above in 'custom_fc_parameters':

    'iat', 'msg_frequency', 'rolling_mean_iat', 'rolling_std_iat', 
    'value__binned_entropy__max_bins_10', 'value__skewness', 
    'value__autocorrelation__lag_1', 'value__agg_autocorrelation__f_agg_"mean"__maxlag_5', 
    'value__agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"', 
    'value__fft_coefficient__attr_"real"__coeff_1', 
    'value__approximate_entropy__m_2__r_0.1', 'value__binned_entropy__max_bins_10', 
    'value__benford_correlation'
"""
# List of selected tsfresh features
tsfresh_features = [
    'iat', 'msg_frequency', 'rolling_mean_iat', 'rolling_std_iat', 
    'value__binned_entropy__max_bins_10', 'value__skewness', 
    'value__autocorrelation__lag_1', 'value__agg_autocorrelation__f_agg_"mean"__maxlag_5', 
    'value__agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"mean"', 
    'value__fft_coefficient__attr_"real"__coeff_1', 
    'value__approximate_entropy__m_2__r_0.1', 'value__binned_entropy__max_bins_10', 
    'value__benford_correlation'
]

In [ ]:
reload(utils)
LOAD_DATAFRAME = False
TS_FRESH = False

n_rows_train = 70000    # select how many rows to load. None if whole train datasset
n_rows_test = 25000   # select how many rows to load. None if whole test datasset
batch_size = 1024
window_size = 50    # increase window size
stride = 5           # increase stride as a buffer
split_ratio = 0.8     # % of training data to use for training
window_anomaly_ratio = 0.1   # For 1 anomaly per window do: 1 / window_size
if LOAD_DATAFRAME:
    # Load training data
    processeddataframe = pd.read_pickle(processeddataframe_normal_pickle_path)
    train_dataset = convert_to_tensorflow(processeddataframe['features'], batch_size= batch_size)

    # Load test data
    processeddataframe_test = pd.read_pickle(processeddataframe_DoS_pickle_path)
    test_dataset = convert_to_tensorflow(processeddataframe_test['features'] ,processeddataframe_test['type'], batch_size= batch_size )
else:
    # Preprocess and load training data
    processeddataframe = feature_selection_preparation(preprocessed_normal_channel2_csv_path, 'training', rows=n_rows_train, ts_fresh= TS_FRESH, ts_fresh_parameters= tsfresh_features, ts_fresh_custom_features= custom_fc_parameters)
    train_dataset, val_dataset = convert_to_tensorflow(processeddataframe['features'], batch_size= batch_size, window_size = window_size, stride = stride, split_ratio= split_ratio)

    processeddataframe_test = feature_selection_preparation(preprocessed_new_Spoofing_channel2_csv_path, 'test', rows=n_rows_test, ts_fresh= TS_FRESH, ts_fresh_parameters= tsfresh_features, ts_fresh_custom_features= custom_fc_parameters)
    test_dataset, test_threshold_dataset = convert_to_tensorflow(processeddataframe_test['features'], processeddataframe_test['type'], batch_size = batch_size, window_size= window_size, stride=stride, window_anomaly_ratio = window_anomaly_ratio)

    #processeddataframe_dos_baseline = feature_selection_preparation(preprocessed_DoS_channel2_csv_path, 'test', rows=n_rows_test, ts_fresh= TS_FRESH, ts_fresh_parameters= tsfresh_features, ts_fresh_custom_features= custom_fc_parameters)
    #dos_test_dataset, dos_threshold_dataset = convert_to_tensorflow(processeddataframe_dos_baseline['features'], processeddataframe_dos_baseline['type'], batch_size = batch_size, window_size= window_size, stride=stride, window_anomaly_ratio = window_anomaly_ratio)


**SAVE Dataset to npz**

In [ ]:
path=r"/Users/SCHUGD/Desktop/MasterThesis/Dataset/Processed_Dataframes/Channel2_Full_43Features_WindowFeatures_1024Batch_50Window_5Stride.npz"

# Save your datasets
save_datasets(train_dataset, val_dataset, path)

# Later, load them back
train_dataset_loaded, val_dataset_loaded = load_datasets(path)

In [ ]:
#print(processeddataframe.head(1))
print(processeddataframe['features'].head(4))
print(np.size(processeddataframe['features'][0]))
# todo: check that all values normalized or there's nans in the sliding windows
# todo: check whole processing of df
# Assuming 'df' is your DataFrame
processeddataframe['has_nan'] = processeddataframe['features'].apply(lambda x: any(pd.isna(x)) if isinstance(x, list) else np.nan)

# Ensure all values in 'features' are lists before flattening
valid_lists = [lst for lst in processeddataframe['features'] if isinstance(lst, list)]

# Flatten the valid lists
all_values = sum(valid_lists, [])

# Check for NaN values
has_nan = any(pd.isna(all_values))

# Check for out-of-bounds values
has_out_of_bounds = any(x < 0 or x > 1 for x in all_values if isinstance(x, (int, float)))

print(f"Contains NaN: {has_nan}")
print(f"Contains values <0 or >1: {has_out_of_bounds}")

def check_dataset(dataset, dataset_name="Dataset"):
    all_values = []
    
    for batch in dataset:
        # Convert TensorFlow tensors to NumPy
        batch_np = batch.numpy() if isinstance(batch, tf.Tensor) else np.array(batch)
        
        # Flatten batch and store values
        all_values.extend(batch_np.flatten())
        #print(batch_np.flatten())
        #print(len(all_values))
    
    # Perform checks
    has_nan = np.isnan(all_values).any()
    has_out_of_bounds = (np.array(all_values) < 0).any() or (np.array(all_values) > 1).any()
    all_values = [lst if isinstance(lst, (list, np.ndarray)) else [lst] for lst in all_values]
    all_same_size = all(len(lst) == len(all_values[0]) for lst in all_values)
    # Final condition combining both checks
    if not all_same_size:
        print("Error: Lists have different sizes.")
    print(f"{dataset_name} - Contains NaN: {has_nan}")
    print(f"{dataset_name} - Contains values <0 or >1: {has_out_of_bounds}")

# Run checks for both datasets
check_dataset(train_dataset, "Train Dataset")
check_dataset(val_dataset, "Validation Dataset")

In [ ]:
print(processeddataframe.head(10))

-----------------------------------------------------------

**VAE SETUP**

In [ ]:
@keras.saving.register_keras_serializable(package="VAE_multiplesamplesOld")
class VAE_multiplesamplesOld(Model):
    def __init__(self, input_dim, latent_dim, window_size, **kwargs):
        super(VAE_multiplesamplesOld, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim)
            layers.LSTM(128, activation='relu', return_sequences=True),  
            layers.BatchNormalization(),
            layers.Dense(64, activation='relu'),
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            layers.Dense(128, activation='relu'),
            layers.Dense(window_size * input_dim, activation='sigmoid'),  
            layers.Reshape((window_size, input_dim))  
        ])

    def encode(self, x):
        hidden = self.encoder(x)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        std = tf.exp(0.5 * logvar)
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)

    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(VAE_multiplesamplesOld, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

In [ ]:
@keras.saving.register_keras_serializable(package="Base_VAE_multiplesamples")
class Base_VAE_multiplesamples(Model):
    def __init__(self, input_dim, latent_dim, window_size, **kwargs):
        super(Base_VAE_multiplesamples, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim) Experiment: HE Init
            layers.LSTM(128, activation='relu', return_sequences=True), 
            layers.BatchNormalization(), # Experiment: BN
            layers.LSTM(64, activation='relu', return_sequences= False),
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder with LSTM layers
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.RepeatVector(window_size),  
            layers.LSTM(64, activation='relu', return_sequences=True),
            layers.BatchNormalization(),
            layers.LSTM(128, activation='relu', return_sequences=True),
            layers.TimeDistributed(layers.Dense(input_dim, activation='sigmoid'))  # Output must match (window_size, input_dim)
        ])

    def encode(self, x):
        hidden = self.encoder(x)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        std = tf.exp(0.5 * logvar)
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)
    

    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    
    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(Base_VAE_multiplesamples, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

In [ ]:
@keras.saving.register_keras_serializable(package="VAE_multiplesamples")
class VAE_multiplesamples(Model):
    def __init__(self, input_dim, latent_dim, window_size ,**kwargs):
        super(VAE_multiplesamples, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        ## Standard VAE ##
        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim) Experiment: HE Init
            layers.LSTM(84, activation='relu', return_sequences=True), 
            #layers.BatchNormalization(), # Experiment: BN
            #layers.LSTM(180, activation='relu', return_sequences= True),
            #layers.LSTM(80, activation='relu', return_sequences= True), # return sequence?
            #layers.Flatten()#layers.flatten 32* 50 , if this is removed u need to uncomment pooling in encode()
            layers.GlobalAveragePooling1D()
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean  make input smaller than latent dim?
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder with LSTM layers                   # Weaker Generator --> Better Encoder --> Better Latent Space?
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.RepeatVector(window_size), 
            layers.LSTM(84, activation = 'relu', return_sequences = True), 
            #layers.LSTM(80, activation='relu', return_sequences=True),
            #layers.BatchNormalization(),
            #layers.LSTM(120, activation='relu', return_sequences=True),
            layers.TimeDistributed(layers.Dense(input_dim, activation='sigmoid'))  # Output must match (window_size, input_dim)
        ])
        #####################################################################################
        ## FactorVAE ##
        # Discriminator for TC estimation
        self.discriminator = tf.keras.Sequential([
                layers.InputLayer(shape=(latent_dim,)),
                #layers.Dense(512),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(256),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(128),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dense(1)
                layers.Dense(2)
                #layers.Dense(1, activation='sigmoid')
            ])
        #####################################################################################

    ## Standard VAE Functions ##
    def encode(self, x):
        hidden = self.encoder(x)
        #hidden = tf.reduce_mean(hidden, axis=1)  # Mean over time (batch_size, hidden_dim) REMOVE
        
        #hidden_pooled = tf.reduce_max(hidden, axis=1)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        logvar = tf.clip_by_value(logvar, -10, 5)  # Prevents extreme values UNSTABLE FIX
        #print(logvar)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        #std = tf.exp(0.5 * logvar)
        std = tf.sqrt(tf.nn.softplus(logvar))  # More stable alternative to exp UNSTABLE FIX
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        
        #print(std)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)
        
    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1, latent_only = False):
        if latent_only:
            mu, logvar = self.encode(x)
            return _ , mu, logvar
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    #####################################################################################
    ## FactorVAE Functions ##
    """
    Following section implements FactorVAE based on:

    """
    def permute_dims(self, z):
        """Permutes the batch dimension to estimate Total Correlation"""
        """Permutes each latent dimension independently to estimate Total Correlation"""
        B, D = tf.shape(z)[0], tf.shape(z)[1]  # Batch size, Latent dimension
        z_perm = tf.zeros_like(z)
        for j in range(D):
            perm = tf.random.shuffle(tf.range(B))
            z_perm = tf.tensor_scatter_nd_update(z_perm, tf.stack([tf.range(B), tf.tile([j], [B])], axis=1), tf.gather(z[:, j], perm))
        return z_perm
    
    def discriminator_loss(self, z, z_perm):
        """Compute discriminator loss for TC estimation using Dense(2)"""
        real_logits = self.discriminator(z)  # Shape: (batch_size, 2)
        fake_logits = self.discriminator(z_perm)  # Shape: (batch_size, 2)
        
        real_labels = tf.zeros((tf.shape(real_logits)[0],), dtype=tf.int32)  # Class 0 for real samples
        fake_labels = tf.ones((tf.shape(fake_logits)[0],), dtype=tf.int32)   # Class 1 for permuted samples
        
        real_loss = tf.keras.losses.sparse_categorical_crossentropy(real_labels, real_logits, from_logits=True)
        fake_loss = tf.keras.losses.sparse_categorical_crossentropy(fake_labels, fake_logits, from_logits=True)
        
        return 0.5 * (tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss))
    
    def tc_loss(self, z):
        """Estimate Total Correlation using the discriminator"""
        logits = self.discriminator(z)
        #logits_perm = self.discriminator(self.permute_dims(z))
        # Density ratio trick
        #tc = tf.reduce_mean(logits_real - logits_perm)
        #tc_2 = tf.reduce_mean(logits[:, :1] - logits[:,1:])
        tc = tf.reduce_mean(logits[:,0] - logits[:,1]) # correct?
        return tc
    
    def discriminator_acc(self, z):
        # Permute dimensions to create "fake" samples
        z_perm = self.permute_dims(z)

        # Get discriminator outputs (logits for two classes)
        logits_real = self.discriminator(z)       # Shape: (batch_size, 2)
        logits_perm = self.discriminator(z_perm)  # Shape: (batch_size, 2)

        # Compute predicted class (0 = real, 1 = permuted)
        preds_real = tf.argmax(logits_real, axis=1)
        preds_perm = tf.argmax(logits_perm, axis=1)

        # Compute accuracy: real samples should be classified as 0, permuted as 1
        acc_real = tf.reduce_mean(tf.cast(tf.equal(preds_real, 0), tf.float32))
        acc_perm = tf.reduce_mean(tf.cast(tf.equal(preds_perm, 1), tf.float32))

        # Average accuracy
        discriminator_accuracy = 0.5 * (acc_real + acc_perm)

        return discriminator_accuracy
    #####################################################################################
    ## β-TCVAE ##
    """
    Following section implements β-TCVAE based on:

    "Isolating Sources of Disentanglement in Variational Autoencoders"
    (https://arxiv.org/pdf/1802.04942).

    if gamma = alpha = 1 , the original function can be rewritten to only
    calculate the Total Correlation similar to FactorVAE
    """

    def gaussian_log_density(self,samples, mean, log_squared_scale):
        pi = tf.constant(np.pi)
        normalization = tf.math.log(2. * pi)
        #inv_sigma = tf.math.exp(-log_squared_scale)
            # Use the same transformation as in reparameterize
        var = tf.nn.softplus(log_squared_scale)  # Match the softplus used in reparameterize
        inv_sigma = 1.0 / var
        tmp = (samples - mean)
        return -0.5 * (tmp * tmp * inv_sigma + log_squared_scale + normalization)

    def b_tcvae_total_correlation_loss(self,z, z_mean, z_log_squared_scale):
        """Estimate of total correlation on a batch.
        We need to compute the expectation over a batch of: E_j [log(q(z(x_j))) -
        log(prod_l q(z(x_j)_l))]. We ignore the constants as they do not matter
        for the minimization. The constant should be equal to (num_latents - 1) *
        log(batch_size * dataset_size)
        Args:
        z: [batch_size, num_latents]-tensor with sampled representation.
        z_mean: [batch_size, num_latents]-tensor with mean of the encoder.
        z_log_squared_scale: [batch_size, num_latents]-tensor with log variance of the encoder.
        Returns:
        Total correlation estimated on a batch.
        """
        #print("Z shape: ", z.shape)
        #print("z_mean shape: ", z_mean.shape)
        #print("z_logvar shape: ", z_log_squared_scale.shape)
        # Compute log(q(z(x_j)|x_i)) for every sample in the batch, which is a
        # tensor of size [batch_size, batch_size, num_latents]. In the following
        log_qz_prob = self.gaussian_log_density(
            tf.expand_dims(z, 1), tf.expand_dims(z_mean, 0),
            tf.expand_dims(z_log_squared_scale, 0))
        #print("[batch_size, batch_size, num_latents]",log_qz_prob.shape)

        # Compute log prod_l p(z(x_j)_l) = sum_l(log(sum_i(q(z(z_j)_l|x_i)))
        # + constant) for each sample in the batch, which is a vector of size
        # [batch_size,].
        log_qz_product = tf.math.reduce_sum(
            tf.math.reduce_logsumexp(log_qz_prob, axis=1, keepdims=False),
            axis=1,
            keepdims=False)
        #print("[batch_size,]",log_qz_product.shape)

        # Compute log(q(z(x_j))) as log(sum_i(q(z(x_j)|x_i))) + constant =
        # log(sum_i(prod_l q(z(x_j)_l|x_i))) + constant.
        log_qz = tf.math.reduce_logsumexp(
            tf.math.reduce_sum(log_qz_prob, axis=2, keepdims=False),
            axis=1,
            keepdims=False)
        return tf.math.reduce_mean(log_qz - log_qz_product)

    #####################################################################################

    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(VAE_multiplesamples, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

In [ ]:
@keras.saving.register_keras_serializable(package="VAE_weak_generator")
class VAE_weakGenerator(Model):
    def __init__(self, input_dim, latent_dim, window_size ,**kwargs):
        super(VAE_weakGenerator, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        ## Standard VAE ##
        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim) Experiment: HE Init
            layers.LSTM(84, activation='relu', return_sequences=True), 
            #layers.BatchNormalization(), # Experiment: BN
            #layers.LSTM(180, activation='relu', return_sequences= True),
            #layers.LSTM(80, activation='relu', return_sequences= True), # return sequence?
            #layers.Flatten()#layers.flatten 32* 50 , if this is removed u need to uncomment pooling in encode()
            layers.GlobalAveragePooling1D()
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean  make input smaller than latent dim?
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder with LSTM layers                   # Weaker Generator --> Better Encoder --> Better Latent Space?
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.RepeatVector(window_size), 
            layers.LSTM(32, activation = 'relu', return_sequences = True), 
            #layers.LSTM(80, activation='relu', return_sequences=True),
            #layers.BatchNormalization(),
            #layers.LSTM(120, activation='relu', return_sequences=True),
            layers.TimeDistributed(layers.Dense(input_dim, activation='sigmoid'))  # Output must match (window_size, input_dim)
        ])
        #####################################################################################
        ## FactorVAE ##
        # Discriminator for TC estimation
        self.discriminator = tf.keras.Sequential([
                layers.InputLayer(shape=(latent_dim,)),
                #layers.Dense(512),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(256),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(128),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dense(1)
                layers.Dense(2)
                #layers.Dense(1, activation='sigmoid')
            ])
        #####################################################################################

    ## Standard VAE Functions ##
    def encode(self, x):
        hidden = self.encoder(x)
        #hidden = tf.reduce_mean(hidden, axis=1)  # Mean over time (batch_size, hidden_dim) REMOVE
        
        #hidden_pooled = tf.reduce_max(hidden, axis=1)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        logvar = tf.clip_by_value(logvar, -10, 5)  # Prevents extreme values UNSTABLE FIX
        #print(logvar)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        #std = tf.exp(0.5 * logvar)
        std = tf.sqrt(tf.nn.softplus(logvar))  # More stable alternative to exp UNSTABLE FIX
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        
        #print(std)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)
        
    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1, latent_only = False):
        if latent_only:
            mu, logvar = self.encode(x)
            return _ , mu, logvar
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    #####################################################################################
    ## FactorVAE Functions ##
    """
    Following section implements FactorVAE based on:

    """
    def permute_dims(self, z):
        """Permutes the batch dimension to estimate Total Correlation"""
        """Permutes each latent dimension independently to estimate Total Correlation"""
        B, D = tf.shape(z)[0], tf.shape(z)[1]  # Batch size, Latent dimension
        z_perm = tf.zeros_like(z)
        for j in range(D):
            perm = tf.random.shuffle(tf.range(B))
            z_perm = tf.tensor_scatter_nd_update(z_perm, tf.stack([tf.range(B), tf.tile([j], [B])], axis=1), tf.gather(z[:, j], perm))
        return z_perm
    
    def discriminator_loss(self, z, z_perm):
        """Compute discriminator loss for TC estimation using Dense(2)"""
        real_logits = self.discriminator(z)  # Shape: (batch_size, 2)
        fake_logits = self.discriminator(z_perm)  # Shape: (batch_size, 2)
        
        real_labels = tf.zeros((tf.shape(real_logits)[0],), dtype=tf.int32)  # Class 0 for real samples
        fake_labels = tf.ones((tf.shape(fake_logits)[0],), dtype=tf.int32)   # Class 1 for permuted samples
        
        real_loss = tf.keras.losses.sparse_categorical_crossentropy(real_labels, real_logits, from_logits=True)
        fake_loss = tf.keras.losses.sparse_categorical_crossentropy(fake_labels, fake_logits, from_logits=True)
        
        return 0.5 * (tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss))
    
    def tc_loss(self, z):
        """Estimate Total Correlation using the discriminator"""
        logits = self.discriminator(z)
        #logits_perm = self.discriminator(self.permute_dims(z))
        # Density ratio trick
        #tc = tf.reduce_mean(logits_real - logits_perm)
        #tc_2 = tf.reduce_mean(logits[:, :1] - logits[:,1:])
        tc = tf.reduce_mean(logits[:,0] - logits[:,1]) # correct?
        return tc
    
    def discriminator_acc(self, z):
        # Permute dimensions to create "fake" samples
        z_perm = self.permute_dims(z)

        # Get discriminator outputs (logits for two classes)
        logits_real = self.discriminator(z)       # Shape: (batch_size, 2)
        logits_perm = self.discriminator(z_perm)  # Shape: (batch_size, 2)

        # Compute predicted class (0 = real, 1 = permuted)
        preds_real = tf.argmax(logits_real, axis=1)
        preds_perm = tf.argmax(logits_perm, axis=1)

        # Compute accuracy: real samples should be classified as 0, permuted as 1
        acc_real = tf.reduce_mean(tf.cast(tf.equal(preds_real, 0), tf.float32))
        acc_perm = tf.reduce_mean(tf.cast(tf.equal(preds_perm, 1), tf.float32))

        # Average accuracy
        discriminator_accuracy = 0.5 * (acc_real + acc_perm)

        return discriminator_accuracy
    #####################################################################################
    ## β-TCVAE ##
    """
    Following section implements β-TCVAE based on:

    "Isolating Sources of Disentanglement in Variational Autoencoders"
    (https://arxiv.org/pdf/1802.04942).

    if gamma = alpha = 1 , the original function can be rewritten to only
    calculate the Total Correlation similar to FactorVAE
    """

    def gaussian_log_density(self,samples, mean, log_squared_scale):
        pi = tf.constant(np.pi)
        normalization = tf.math.log(2. * pi)
        #inv_sigma = tf.math.exp(-log_squared_scale)
            # Use the same transformation as in reparameterize
        var = tf.nn.softplus(log_squared_scale)  # Match the softplus used in reparameterize
        inv_sigma = 1.0 / var
        tmp = (samples - mean)
        return -0.5 * (tmp * tmp * inv_sigma + log_squared_scale + normalization)

    def b_tcvae_total_correlation_loss(self,z, z_mean, z_log_squared_scale):
        """Estimate of total correlation on a batch.
        We need to compute the expectation over a batch of: E_j [log(q(z(x_j))) -
        log(prod_l q(z(x_j)_l))]. We ignore the constants as they do not matter
        for the minimization. The constant should be equal to (num_latents - 1) *
        log(batch_size * dataset_size)
        Args:
        z: [batch_size, num_latents]-tensor with sampled representation.
        z_mean: [batch_size, num_latents]-tensor with mean of the encoder.
        z_log_squared_scale: [batch_size, num_latents]-tensor with log variance of the encoder.
        Returns:
        Total correlation estimated on a batch.
        """
        #print("Z shape: ", z.shape)
        #print("z_mean shape: ", z_mean.shape)
        #print("z_logvar shape: ", z_log_squared_scale.shape)
        # Compute log(q(z(x_j)|x_i)) for every sample in the batch, which is a
        # tensor of size [batch_size, batch_size, num_latents]. In the following
        log_qz_prob = self.gaussian_log_density(
            tf.expand_dims(z, 1), tf.expand_dims(z_mean, 0),
            tf.expand_dims(z_log_squared_scale, 0))
        #print("[batch_size, batch_size, num_latents]",log_qz_prob.shape)

        # Compute log prod_l p(z(x_j)_l) = sum_l(log(sum_i(q(z(z_j)_l|x_i)))
        # + constant) for each sample in the batch, which is a vector of size
        # [batch_size,].
        log_qz_product = tf.math.reduce_sum(
            tf.math.reduce_logsumexp(log_qz_prob, axis=1, keepdims=False),
            axis=1,
            keepdims=False)
        #print("[batch_size,]",log_qz_product.shape)

        # Compute log(q(z(x_j))) as log(sum_i(q(z(x_j)|x_i))) + constant =
        # log(sum_i(prod_l q(z(x_j)_l|x_i))) + constant.
        log_qz = tf.math.reduce_logsumexp(
            tf.math.reduce_sum(log_qz_prob, axis=2, keepdims=False),
            axis=1,
            keepdims=False)
        return tf.math.reduce_mean(log_qz - log_qz_product)

    #####################################################################################

    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(VAE_weakGenerator, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

In [ ]:
@keras.saving.register_keras_serializable(package="VAE_2x")
class VAE_2x(Model):
    def __init__(self, input_dim, latent_dim, window_size ,**kwargs):
        super(VAE_2x, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        ## Standard VAE ##
        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim) Experiment: HE Init
            layers.LSTM(120, activation='relu', return_sequences=True), 
            #layers.BatchNormalization(), # Experiment: BN
            #layers.LSTM(180, activation='relu', return_sequences= True),
            #layers.LSTM(80, activation='relu', return_sequences= True), # return sequence?
            #layers.Flatten()#layers.flatten 32* 50 , if this is removed u need to uncomment pooling in encode()
            layers.GlobalAveragePooling1D()
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean  make input smaller than latent dim?
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder with LSTM layers                   # Weaker Generator --> Better Encoder --> Better Latent Space?
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.RepeatVector(window_size), 
            layers.LSTM(120, activation = 'relu', return_sequences = True), 
            #layers.LSTM(80, activation='relu', return_sequences=True),
            #layers.BatchNormalization(),
            #layers.LSTM(120, activation='relu', return_sequences=True),
            layers.TimeDistributed(layers.Dense(input_dim, activation='sigmoid'))  # Output must match (window_size, input_dim)
        ])
        #####################################################################################
        ## FactorVAE ##
        # Discriminator for TC estimation
        self.discriminator = tf.keras.Sequential([
                layers.InputLayer(shape=(latent_dim,)),
                #layers.Dense(512),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(256),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(128),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dense(1)
                layers.Dense(2)
                #layers.Dense(1, activation='sigmoid')
            ])
        #####################################################################################

    ## Standard VAE Functions ##
    def encode(self, x):
        hidden = self.encoder(x)
        #hidden = tf.reduce_mean(hidden, axis=1)  # Mean over time (batch_size, hidden_dim) REMOVE
        
        #hidden_pooled = tf.reduce_max(hidden, axis=1)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        logvar = tf.clip_by_value(logvar, -10, 5)  # Prevents extreme values UNSTABLE FIX
        #print(logvar)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        #std = tf.exp(0.5 * logvar)
        std = tf.sqrt(tf.nn.softplus(logvar))  # More stable alternative to exp UNSTABLE FIX
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        
        #print(std)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)
        
    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1, latent_only = False):
        if latent_only:
            mu, logvar = self.encode(x)
            return _ , mu, logvar
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    #####################################################################################
    ## FactorVAE Functions ##
    """
    Following section implements FactorVAE based on:

    """
    def permute_dims(self, z):
        """Permutes the batch dimension to estimate Total Correlation"""
        """Permutes each latent dimension independently to estimate Total Correlation"""
        B, D = tf.shape(z)[0], tf.shape(z)[1]  # Batch size, Latent dimension
        z_perm = tf.zeros_like(z)
        for j in range(D):
            perm = tf.random.shuffle(tf.range(B))
            z_perm = tf.tensor_scatter_nd_update(z_perm, tf.stack([tf.range(B), tf.tile([j], [B])], axis=1), tf.gather(z[:, j], perm))
        return z_perm
    
    def discriminator_loss(self, z, z_perm):
        """Compute discriminator loss for TC estimation using Dense(2)"""
        real_logits = self.discriminator(z)  # Shape: (batch_size, 2)
        fake_logits = self.discriminator(z_perm)  # Shape: (batch_size, 2)
        
        real_labels = tf.zeros((tf.shape(real_logits)[0],), dtype=tf.int32)  # Class 0 for real samples
        fake_labels = tf.ones((tf.shape(fake_logits)[0],), dtype=tf.int32)   # Class 1 for permuted samples
        
        real_loss = tf.keras.losses.sparse_categorical_crossentropy(real_labels, real_logits, from_logits=True)
        fake_loss = tf.keras.losses.sparse_categorical_crossentropy(fake_labels, fake_logits, from_logits=True)
        
        return 0.5 * (tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss))
    
    def tc_loss(self, z):
        """Estimate Total Correlation using the discriminator"""
        logits = self.discriminator(z)
        #logits_perm = self.discriminator(self.permute_dims(z))
        # Density ratio trick
        #tc = tf.reduce_mean(logits_real - logits_perm)
        #tc_2 = tf.reduce_mean(logits[:, :1] - logits[:,1:])
        tc = tf.reduce_mean(logits[:,0] - logits[:,1]) # correct?
        return tc
    
    def discriminator_acc(self, z):
        # Permute dimensions to create "fake" samples
        z_perm = self.permute_dims(z)

        # Get discriminator outputs (logits for two classes)
        logits_real = self.discriminator(z)       # Shape: (batch_size, 2)
        logits_perm = self.discriminator(z_perm)  # Shape: (batch_size, 2)

        # Compute predicted class (0 = real, 1 = permuted)
        preds_real = tf.argmax(logits_real, axis=1)
        preds_perm = tf.argmax(logits_perm, axis=1)

        # Compute accuracy: real samples should be classified as 0, permuted as 1
        acc_real = tf.reduce_mean(tf.cast(tf.equal(preds_real, 0), tf.float32))
        acc_perm = tf.reduce_mean(tf.cast(tf.equal(preds_perm, 1), tf.float32))

        # Average accuracy
        discriminator_accuracy = 0.5 * (acc_real + acc_perm)

        return discriminator_accuracy
    #####################################################################################
    ## β-TCVAE ##
    """
    Following section implements β-TCVAE based on:

    "Isolating Sources of Disentanglement in Variational Autoencoders"
    (https://arxiv.org/pdf/1802.04942).

    if gamma = alpha = 1 , the original function can be rewritten to only
    calculate the Total Correlation similar to FactorVAE
    """

    def gaussian_log_density(self,samples, mean, log_squared_scale):
        pi = tf.constant(np.pi)
        normalization = tf.math.log(2. * pi)
        #inv_sigma = tf.math.exp(-log_squared_scale)
            # Use the same transformation as in reparameterize
        var = tf.nn.softplus(log_squared_scale)  # Match the softplus used in reparameterize
        inv_sigma = 1.0 / var
        tmp = (samples - mean)
        return -0.5 * (tmp * tmp * inv_sigma + log_squared_scale + normalization)

    def b_tcvae_total_correlation_loss(self,z, z_mean, z_log_squared_scale):
        """Estimate of total correlation on a batch.
        We need to compute the expectation over a batch of: E_j [log(q(z(x_j))) -
        log(prod_l q(z(x_j)_l))]. We ignore the constants as they do not matter
        for the minimization. The constant should be equal to (num_latents - 1) *
        log(batch_size * dataset_size)
        Args:
        z: [batch_size, num_latents]-tensor with sampled representation.
        z_mean: [batch_size, num_latents]-tensor with mean of the encoder.
        z_log_squared_scale: [batch_size, num_latents]-tensor with log variance of the encoder.
        Returns:
        Total correlation estimated on a batch.
        """
        #print("Z shape: ", z.shape)
        #print("z_mean shape: ", z_mean.shape)
        #print("z_logvar shape: ", z_log_squared_scale.shape)
        # Compute log(q(z(x_j)|x_i)) for every sample in the batch, which is a
        # tensor of size [batch_size, batch_size, num_latents]. In the following
        log_qz_prob = self.gaussian_log_density(
            tf.expand_dims(z, 1), tf.expand_dims(z_mean, 0),
            tf.expand_dims(z_log_squared_scale, 0))
        #print("[batch_size, batch_size, num_latents]",log_qz_prob.shape)

        # Compute log prod_l p(z(x_j)_l) = sum_l(log(sum_i(q(z(z_j)_l|x_i)))
        # + constant) for each sample in the batch, which is a vector of size
        # [batch_size,].
        log_qz_product = tf.math.reduce_sum(
            tf.math.reduce_logsumexp(log_qz_prob, axis=1, keepdims=False),
            axis=1,
            keepdims=False)
        #print("[batch_size,]",log_qz_product.shape)

        # Compute log(q(z(x_j))) as log(sum_i(q(z(x_j)|x_i))) + constant =
        # log(sum_i(prod_l q(z(x_j)_l|x_i))) + constant.
        log_qz = tf.math.reduce_logsumexp(
            tf.math.reduce_sum(log_qz_prob, axis=2, keepdims=False),
            axis=1,
            keepdims=False)
        return tf.math.reduce_mean(log_qz - log_qz_product)

    #####################################################################################

    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(VAE_2x, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

In [ ]:
@keras.saving.register_keras_serializable(package="VAE_2x_weak_generator")
class VAE_2x_weak_generator(Model):
    def __init__(self, input_dim, latent_dim, window_size ,**kwargs):
        super(VAE_2x_weak_generator, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.window_size = window_size  

        ## Standard VAE ##
        # Encoder
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(shape=(window_size, input_dim)),  # Sequence input shape (window_size, input_dim) Experiment: HE Init
            layers.LSTM(120, activation='relu', return_sequences=True), 
            #layers.BatchNormalization(), # Experiment: BN
            #layers.LSTM(180, activation='relu', return_sequences= True),
            #layers.LSTM(80, activation='relu', return_sequences= True), # return sequence?
            #layers.Flatten()#layers.flatten 32* 50 , if this is removed u need to uncomment pooling in encode()
            layers.GlobalAveragePooling1D()
        ])
        self.fc_mu = layers.Dense(latent_dim)  # Latent mean  make input smaller than latent dim?
        self.fc_logvar = layers.Dense(latent_dim)  # Latent log variance

        # Decoder with LSTM layers                   # Weaker Generator --> Better Encoder --> Better Latent Space?
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.RepeatVector(window_size), 
            layers.LSTM(32, activation = 'relu', return_sequences = True), 
            #layers.LSTM(80, activation='relu', return_sequences=True),
            #layers.BatchNormalization(),
            #layers.LSTM(120, activation='relu', return_sequences=True),
            layers.TimeDistributed(layers.Dense(input_dim, activation='sigmoid'))  # Output must match (window_size, input_dim)
        ])
        #####################################################################################
        ## FactorVAE ##
        # Discriminator for TC estimation
        self.discriminator = tf.keras.Sequential([
                layers.InputLayer(shape=(latent_dim,)),
                #layers.Dense(512),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(256),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                
                #layers.Dense(128),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dropout(0.2),
                #layers.BatchNormalization(),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                layers.Dense(1500),
                layers.LeakyReLU(0.2),
                #layers.Dense(1)
                layers.Dense(2)
                #layers.Dense(1, activation='sigmoid')
            ])
        #####################################################################################

    ## Standard VAE Functions ##
    def encode(self, x):
        hidden = self.encoder(x)
        #hidden = tf.reduce_mean(hidden, axis=1)  # Mean over time (batch_size, hidden_dim) REMOVE
        
        #hidden_pooled = tf.reduce_max(hidden, axis=1)
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        logvar = tf.clip_by_value(logvar, -10, 5)  # Prevents extreme values UNSTABLE FIX
        #print(logvar)
        return mu, logvar

    def reparameterize(self, mu, logvar, n_samples=1):
        eps = tf.random.normal(shape=(n_samples, tf.shape(mu)[0], tf.shape(mu)[1]))
        #std = tf.exp(0.5 * logvar)
        std = tf.sqrt(tf.nn.softplus(logvar))  # More stable alternative to exp UNSTABLE FIX
        mu = tf.expand_dims(mu, axis=0)  # Expand for n_samples
        std = tf.expand_dims(std, axis=0)
        
        #print(std)
        return mu + eps * std  # Shape: (n_samples, batch_size, latent_dim)
        
    def decode(self, z):
        # Flatten the samples for decoding
        z_flat = tf.reshape(z, (-1, tf.shape(z)[-1]))  # (n_samples * batch_size, latent_dim)
        reconstructed_flat = self.decoder(z_flat)
        # Reshape back to (n_samples, batch_size, window_size, input_dim)
        return tf.reshape(reconstructed_flat, (tf.shape(z)[0], tf.shape(z)[1], self.window_size, self.input_dim))

    def call(self, x, n_samples=1, latent_only = False):
        if latent_only:
            mu, logvar = self.encode(x)
            return _ , mu, logvar
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar, n_samples=n_samples)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    #####################################################################################
    ## FactorVAE Functions ##
    """
    Following section implements FactorVAE based on:

    """
    def permute_dims(self, z):
        """Permutes the batch dimension to estimate Total Correlation"""
        """Permutes each latent dimension independently to estimate Total Correlation"""
        B, D = tf.shape(z)[0], tf.shape(z)[1]  # Batch size, Latent dimension
        z_perm = tf.zeros_like(z)
        for j in range(D):
            perm = tf.random.shuffle(tf.range(B))
            z_perm = tf.tensor_scatter_nd_update(z_perm, tf.stack([tf.range(B), tf.tile([j], [B])], axis=1), tf.gather(z[:, j], perm))
        return z_perm
    
    def discriminator_loss(self, z, z_perm):
        """Compute discriminator loss for TC estimation using Dense(2)"""
        real_logits = self.discriminator(z)  # Shape: (batch_size, 2)
        fake_logits = self.discriminator(z_perm)  # Shape: (batch_size, 2)
        
        real_labels = tf.zeros((tf.shape(real_logits)[0],), dtype=tf.int32)  # Class 0 for real samples
        fake_labels = tf.ones((tf.shape(fake_logits)[0],), dtype=tf.int32)   # Class 1 for permuted samples
        
        real_loss = tf.keras.losses.sparse_categorical_crossentropy(real_labels, real_logits, from_logits=True)
        fake_loss = tf.keras.losses.sparse_categorical_crossentropy(fake_labels, fake_logits, from_logits=True)
        
        return 0.5 * (tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss))
    
    def tc_loss(self, z):
        """Estimate Total Correlation using the discriminator"""
        logits = self.discriminator(z)
        #logits_perm = self.discriminator(self.permute_dims(z))
        # Density ratio trick
        #tc = tf.reduce_mean(logits_real - logits_perm)
        #tc_2 = tf.reduce_mean(logits[:, :1] - logits[:,1:])
        tc = tf.reduce_mean(logits[:,0] - logits[:,1]) # correct?
        return tc
    
    def discriminator_acc(self, z):
        # Permute dimensions to create "fake" samples
        z_perm = self.permute_dims(z)

        # Get discriminator outputs (logits for two classes)
        logits_real = self.discriminator(z)       # Shape: (batch_size, 2)
        logits_perm = self.discriminator(z_perm)  # Shape: (batch_size, 2)

        # Compute predicted class (0 = real, 1 = permuted)
        preds_real = tf.argmax(logits_real, axis=1)
        preds_perm = tf.argmax(logits_perm, axis=1)

        # Compute accuracy: real samples should be classified as 0, permuted as 1
        acc_real = tf.reduce_mean(tf.cast(tf.equal(preds_real, 0), tf.float32))
        acc_perm = tf.reduce_mean(tf.cast(tf.equal(preds_perm, 1), tf.float32))

        # Average accuracy
        discriminator_accuracy = 0.5 * (acc_real + acc_perm)

        return discriminator_accuracy
    #####################################################################################
    ## β-TCVAE ##
    """
    Following section implements β-TCVAE based on:

    "Isolating Sources of Disentanglement in Variational Autoencoders"
    (https://arxiv.org/pdf/1802.04942).

    if gamma = alpha = 1 , the original function can be rewritten to only
    calculate the Total Correlation similar to FactorVAE
    """

    def gaussian_log_density(self,samples, mean, log_squared_scale):
        pi = tf.constant(np.pi)
        normalization = tf.math.log(2. * pi)
        #inv_sigma = tf.math.exp(-log_squared_scale)
            # Use the same transformation as in reparameterize
        var = tf.nn.softplus(log_squared_scale)  # Match the softplus used in reparameterize
        inv_sigma = 1.0 / var
        tmp = (samples - mean)
        return -0.5 * (tmp * tmp * inv_sigma + log_squared_scale + normalization)

    def b_tcvae_total_correlation_loss(self,z, z_mean, z_log_squared_scale):
        """Estimate of total correlation on a batch.
        We need to compute the expectation over a batch of: E_j [log(q(z(x_j))) -
        log(prod_l q(z(x_j)_l))]. We ignore the constants as they do not matter
        for the minimization. The constant should be equal to (num_latents - 1) *
        log(batch_size * dataset_size)
        Args:
        z: [batch_size, num_latents]-tensor with sampled representation.
        z_mean: [batch_size, num_latents]-tensor with mean of the encoder.
        z_log_squared_scale: [batch_size, num_latents]-tensor with log variance of the encoder.
        Returns:
        Total correlation estimated on a batch.
        """
        #print("Z shape: ", z.shape)
        #print("z_mean shape: ", z_mean.shape)
        #print("z_logvar shape: ", z_log_squared_scale.shape)
        # Compute log(q(z(x_j)|x_i)) for every sample in the batch, which is a
        # tensor of size [batch_size, batch_size, num_latents]. In the following
        log_qz_prob = self.gaussian_log_density(
            tf.expand_dims(z, 1), tf.expand_dims(z_mean, 0),
            tf.expand_dims(z_log_squared_scale, 0))
        #print("[batch_size, batch_size, num_latents]",log_qz_prob.shape)

        # Compute log prod_l p(z(x_j)_l) = sum_l(log(sum_i(q(z(z_j)_l|x_i)))
        # + constant) for each sample in the batch, which is a vector of size
        # [batch_size,].
        log_qz_product = tf.math.reduce_sum(
            tf.math.reduce_logsumexp(log_qz_prob, axis=1, keepdims=False),
            axis=1,
            keepdims=False)
        #print("[batch_size,]",log_qz_product.shape)

        # Compute log(q(z(x_j))) as log(sum_i(q(z(x_j)|x_i))) + constant =
        # log(sum_i(prod_l q(z(x_j)_l|x_i))) + constant.
        log_qz = tf.math.reduce_logsumexp(
            tf.math.reduce_sum(log_qz_prob, axis=2, keepdims=False),
            axis=1,
            keepdims=False)
        return tf.math.reduce_mean(log_qz - log_qz_product)

    #####################################################################################

    
    def get_config(self):
        # Return the config of the model including custom parameters
        config = super(VAE_2x_weak_generator, self).get_config()
        config.update({
            'input_dim': self.input_dim,  # Input dimension
            'latent_dim': self.fc_mu.units,  # Latent dimension
            'window_size': self.window_size  # Window size
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create the instance from config
        return cls(input_dim=config['input_dim'], latent_dim=config['latent_dim'], window_size=config['window_size'])

Training loop function

In [ ]:
def train_model(vae,optimizer,discriminator_optimizer, epochs, n_samples, input_dim, latent_dim, batch_size,beta, gamma, n_critic, steps_anneal, patience, time,beta_tc = 0, validation_method = 'None', model_path = ""):   
    wait = 0  
    model_name ="LSTM_VAE"

    print(f"Latent Dimension = {latent_dim}, "
        f"Beta = {beta}, Gamma = {gamma}, N_critic = {n_critic}, Beta_TC = {beta_tc}, Validation Method = {validation_method}, "
        f"Rows in Training Data = {n_rows_train}, "
        f"Batch Size = {batch_size}") 
    train_losses = []
    val_losses = []
    disc_losses = []
    beta_tc_losses = []
    best_val_loss = float('inf')
    real_epochs = 0

    # Training loop
    for epoch in range(epochs):
        real_epochs += 1
        epoch_loss = 0
        epoch_disc_loss = 0
        epoch_beta_tc_loss = 0
        for step, batch in enumerate(train_dataset): # window size , features
            global_step = epoch * len(train_dataset) + step  # Total training step count
            # First get latent representations
            #_, mu, logvar = vae(batch, n_samples=1, latent_only=True)
            #z = vae.reparameterize(mu, logvar, n_samples=1)
            #z = tf.squeeze(z, axis=0)
            #print(batch.shape)

            # Split the batch according to FactorVAE Paper
            anneal_coeff = linear_annealing(0, 1, global_step, steps_anneal)

            if gamma > 0 and n_critic > 0:

                half_batch_size = tf.shape(batch)[0] // 2
                first_half_batch = batch[:half_batch_size]
                second_half_batch = batch[half_batch_size:]

                # Compute Annealing Coefficient
                #print(anneal_coeff)
            else:
                first_half_batch = batch

            # 1. Train VAE
            with tf.GradientTape() as tape:
                reconstructed, mu, logvar = vae(first_half_batch, n_samples=n_samples, latent_only = False)  # Use multiple samples

                #Compute reconstruction MSE error for each sample
                reconstruction_errors = tf.reduce_mean(
                    tf.square(tf.expand_dims(first_half_batch, axis=0) - reconstructed), axis=-1
                )  # Shape: (n_samples, batch_size, window_size)

                # Aggregate errors (mean over samples and window_size)
                mean_reconstruction_error = tf.reduce_mean(reconstruction_errors, axis=(2, 0))

                # Compute VAE loss
                reconstruction_loss = tf.reduce_mean(mean_reconstruction_error)

                #print(f"Final loss {reconstruction_loss}")
                kl_divergence = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))
                                
                if gamma > 0:
                    # Get latent samples for TC loss
                    z = vae.reparameterize(mu, logvar, n_samples=n_samples)
                    z = tf.reduce_mean(z, axis=0)  # Remove sample dimension for discriminator
                    
                    # Total Correlation loss
                    tc = vae.tc_loss(z)
                    loss = reconstruction_loss + beta * kl_divergence + gamma*tc*anneal_coeff
                elif beta_tc > 0:
                    z = vae.reparameterize(mu, logvar, n_samples=n_samples)
                    z = tf.reduce_mean(z, 0) # aggregate over latent samples
                    #print("z shape directly after reparam: ", z.shape)
                    tc_beta_loss = vae.b_tcvae_total_correlation_loss(z, mu, logvar)
                    b_tcvae_loss = (1 - beta_tc) * tc_beta_loss
                    #print("tc beta" ,tc_beta_loss.numpy())
                    #print("tc beta * b_tc", b_tcvae_loss.numpy())
                    #print("anneal" , anneal_coeff.numpy())
                    #print(" tc beta * anneal", (tc_beta_loss * anneal_coeff).numpy())

                    #print("recon", reconstruction_loss.numpy() )
                    #print("kl", kl_divergence.numpy())
                    loss = reconstruction_loss + kl_divergence + b_tcvae_loss
                else:
                    #print(reconstruction_loss)
                    #print(kl_divergence)
                    loss = reconstruction_loss + beta * kl_divergence



            gradients = tape.gradient(loss, vae.encoder.trainable_variables + vae.decoder.trainable_variables)
            gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
            optimizer.apply_gradients(zip(gradients, vae.encoder.trainable_variables + vae.decoder.trainable_variables))

            epoch_loss += loss.numpy()
            #epoch_beta_tc_loss += b_tcvae_loss.numpy()
            # 2. Train discriminator
            disc_loss = 0
            if gamma > 0 and n_critic >0:
                for _ in range(n_critic):
                    # Discriminator update
                    with tf.GradientTape() as disc_tape:

                        # First get latent representations on second half batch
                        _, mu, logvar = vae(second_half_batch, n_samples=n_samples, latent_only=True)
                        z = vae.reparameterize(mu, logvar, n_samples=n_samples)
                        z = tf.reduce_mean(z, axis=0)
                        z_perm = vae.permute_dims(z)
                        current_disc_loss = vae.discriminator_loss(z, z_perm)
                        disc_loss += current_disc_loss

                    # Update discriminator
                    disc_gradients = disc_tape.gradient(current_disc_loss, vae.discriminator.trainable_variables)
                    # Clip gradients for stability
                    disc_gradients, _ = tf.clip_by_global_norm(disc_gradients, 1.0)
                    discriminator_optimizer.apply_gradients(zip(disc_gradients, vae.discriminator.trainable_variables))

                disc_loss = disc_loss / n_critic
                epoch_disc_loss += disc_loss
        
        # VALIDATION
        epoch_val_loss = 0
        total_disc_acc = 0
        show_val = validation_method in ["B_VAE","B_TCVAE", "TC"]
        latent_only = validation_method == "TC"

        if validation_method == "None" or validation_method == "PLOT":
            val_loss = 0
            avg_disc_acc = 0
            early_stop = False
        else:
            early_stop = True
            for batch in val_dataset:
                reconstructed, mu, logvar = vae(batch, n_samples=n_samples, latent_only=latent_only)

                if validation_method == "B_VAE":
                    reconstruction_errors = tf.reduce_mean(
                        tf.square(tf.expand_dims(batch, axis=0) - reconstructed), axis=-1
                    )
                    mean_reconstruction_error = tf.reduce_mean(reconstruction_errors, axis=(2, 0))
                    reconstruction_loss = tf.reduce_mean(mean_reconstruction_error)
                    kl_divergence = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))

                    val_loss = reconstruction_loss + beta * kl_divergence
                    epoch_val_loss += val_loss.numpy()
                elif validation_method == "B_TCVAE":
                    reconstruction_errors = tf.reduce_mean(
                        tf.square(tf.expand_dims(batch, axis=0) - reconstructed), axis=-1
                    )
                    mean_reconstruction_error = tf.reduce_mean(reconstruction_errors, axis=(2, 0))
                    reconstruction_loss = tf.reduce_mean(mean_reconstruction_error)
                    kl_divergence = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))

                    z = vae.reparameterize(mu, logvar, n_samples=n_samples)
                    z = tf.reduce_mean(z, axis= 0)
                    tc_beta_loss = vae.b_tcvae_total_correlation_loss(z, mu, logvar)
                    b_tcvae_loss = (1 - beta_tc) * tc_beta_loss
                    
                    val_loss = reconstruction_loss + kl_divergence + b_tcvae_loss

                    epoch_val_loss += val_loss.numpy()
                elif validation_method == "TC":
                    z = vae.reparameterize(mu, logvar, n_samples=n_samples)
                    z = tf.reduce_mean(z, axis= 0)
                    tc = vae.tc_loss(z)
                    
                    disc_acc = vae.discriminator_acc(z)
                    #disc_acc = 0  # Assuming this is optional or calculated elsewhere
                    total_disc_acc += disc_acc
                    val_loss = gamma * tc
                    epoch_val_loss += val_loss.numpy()

        if validation_method == "PLOT" and epoch % 50 == 0 and epoch > 0:
            print("PLOT AT EPOCH: ", {epoch + 1})
            get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,type = 'TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)


        # Store the loss for this epoch
        train_loss = epoch_loss / len(train_dataset)
        val_loss = epoch_val_loss / len(val_dataset) 
        disc_loss = epoch_disc_loss / len(train_dataset)
        avg_disc_acc = total_disc_acc / len(val_dataset)
        beta_tc_loss = epoch_beta_tc_loss / len(train_dataset)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        disc_losses.append(disc_loss)
        beta_tc_losses.append(beta_tc_loss)

        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.6f}, "
            f"Discriminator Loss: {disc_loss:.6f} Disc Acc: {avg_disc_acc:.4f}, Beta TC Loss: {beta_tc_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        if early_stop:
            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                wait = 0
                # Save best model
                best_model_path = model_path
                vae.compile(optimizer = optimizer)
                vae.save(best_model_path)
            else:
                wait += 1
                if wait >= patience:
                    epochs = epoch + 1
                    print(f"Early stopping triggered at epoch {epoch + 1}")
                    break
        
        # Save the trained model
        #vae.compile(optimizer = optimizer)
        #vae.save(model_path)
        if epoch % 10 == 0 and epoch > 0:
            save_trained_model(vae, optimizer, model_path, model_name, latent_dim,beta,n_rows_train,time, AWS = AWS, s3 = s3 , BUCKET = BUCKET)
            print( f'💽Saved Model at Epoch {epoch+1}💽')

    #Save the trained model
    #vae.save(model_path)

    print("💽Loss Plot Saved.💽")
    print("💽VAE model saved.💽")
    #vae.summary()
    return train_losses, val_losses, real_epochs, time, show_val, model_path, vae



--------------------------------------

----------------------------------------------------

**TRAINING**

In [ ]:
reload(utils)
reload(anomaly_detection_functions)
# Regular HYPERPARAMETERS 
input_dim = np.size(processeddataframe['features'][0])
#input_dim = 42
epochs = 2
n_samples = 1
# Best 512 settigns: AdamW with LR=1e-05, WD=1e-06, Beta1=0.85, Beta2=0.98  
learning_rate = 1e-03
weight_decay = 1e-06
# FactorVAE
learning_rate_disc = 5e-5
# Annealing and Early stop
steps_anneal = epochs * len(train_dataset)  
alpha = 0.0  # Minimum learning rate as a fraction of the initial learning rate
validation_method = "None" # None, B_VAE, TC, B_TCVAE
patience = 5  

# IMPORTANT 
latent_dim= 5  
beta = 1 # 20
beta_tc = 0 #1.008 # keep in mind this is tuned based on that we get tc loss = -244..

gamma = 0  # TC weight (typically between 1 and 10) 6
n_critic = 0
##################

AD = False

time = datetime.now().strftime("%H-%M")
model_name ="LSTM_VAE"
model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_{model_name}_LD{latent_dim}_Beta{beta}_NT{n_rows_train}_{time}.keras'

cosine_decay_schedule = CosineDecay(
    1e-03, steps_anneal, alpha=alpha
)


vae = VAE_multiplesamples(input_dim=input_dim, latent_dim=latent_dim, window_size= window_size)

#optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay, beta_1=beta1, beta_2=beta2)
optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate_disc)

train_losses, val_losses, real_epochs, time, show_val, model_path, vae = train_model(vae,optimizer,discriminator_optimizer, epochs,
                                                                    n_samples, input_dim, latent_dim, batch_size,beta,
                                                                      gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                      model_path= model_path)


plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time,n_rows_train, show_val= show_val, AWS = AWS, s3 = s3, BUCKET = BUCKET)
#analyze_latent_variance(vae,train_dataset, test_dataset)
#analyze_kl_divergence(vae, train_dataset, test_dataset)
#get_latent_representations_label(vae, test_dataset,latent_dim, beta,n_critic,gamma,time, 'PCA', save = False)
get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,'TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)
#get_latent_representations_label(vae, train_dataset, 'TSNE')

if AD:
  reconstruction_AD = True
  latent_AD = True
  reconstruction_threshold, latent_threshold, mean_train, variance_train = get_threshold_from_train(model_path,reconstruction_AD, latent_AD)
  results, distances = anomaly_detection(vae, reconstruction_AD, latent_AD, mean_train, variance_train)
  reconstruction_accuracy , latent_accuracy = get_anomaly_detection_accuracy(reconstruction_AD, latent_AD, results,reconstruction_threshold,distances,latent_threshold,model_name, latent_dim,epochs,time, AWS = AWS, s3=s3, BUCKET = BUCKET)

**Test Saved Model**

In [ ]:
#best_model_path = "/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_LSTM_VAE_LD30_Beta25_NT800000_21-37.keras"
#loaded_vae = keras.models.load_model("/Users/SCHUGD/Desktop/MasterThesis/Code/Models/Iter_BEST_BEST_VAE_11-15.keras")
loaded_vae = keras.models.load_model(model_path)
loaded_vae.trainable = False  # Freeze model weights

get_latent_representations_label(loaded_vae, test_dataset, latent_dim, beta,n_critic,gamma,time,'TSNE', save = False)

**HyperParamter Search**

In [ ]:
reload(utils)
reload(anomaly_detection_functions)
# Regular HYPERPARAMETERS 
input_dim = np.size(processeddataframe['features'][0])
#input_dim = 42
epochs = 80
n_samples = 1
# Best 512 settigns: AdamW with LR=1e-05, WD=1e-06, Beta1=0.85, Beta2=0.98  
learning_rate = 1e-04
weight_decay = 1e-06
# FactorVAE
learning_rate_disc = 5e-5
# Annealing and Early stop
steps_anneal = epochs * len(train_dataset)  
alpha = 0.0  # Minimum learning rate as a fraction of the initial learning rate
validation_method = "None" # None, B_VAE, TC, B_TCVAE
patience = 5  

# IMPORTANT 
latent_dim= 2  
beta = 1 # 20
beta_tc = 0 #1.008 # keep in mind this is tuned based on that we get tc loss = -244..

gamma = 0  # TC weight (typically between 1 and 10) 6
n_critic = 0
##################

AD = False

cosine_decay_schedule = CosineDecay(
    1e-03, steps_anneal, alpha=alpha
)

# Hyperparameter search space
latent_dims = [9,14, 20,25]  # Example values for latent dimension
beta_values = [1, 7, 15,30]


# Iterate over all combinations
for latent_dim, beta in itertools.product(latent_dims, beta_values):
    time = datetime.now().strftime("%H-%M")
    model_name ="BASE_LSTM_VAE"
    model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_{model_name}_LD{latent_dim}_Beta{beta}_NT{n_rows_train}_{time}.keras'

    print(f"Training with: latent_dim={latent_dim}, beta={beta} validation_method={validation_method}")

    vae = VAE_multiplesamples(input_dim=input_dim, latent_dim=latent_dim, window_size= window_size)
    vae_weakGen = VAE_weakGenerator(input_dim=input_dim, latent_dim=latent_dim, window_size= window_size)
    vae_2x = VAE_2x(input_dim=input_dim, latent_dim=latent_dim, window_size= window_size)
    VAE_2x_weakGen = VAE_2x_weak_generator(input_dim=input_dim, latent_dim=latent_dim, window_size= window_size)

        #optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay, beta_1=beta1, beta_2=beta2)
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    optimizer_weakGen = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    optimizer_2x = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    optimizer_2x_weakGen = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate_disc)

    print("VAE--------------")
    train_losses, val_losses, real_epochs, time, show_val, model_path, vae = train_model(vae,optimizer,discriminator_optimizer, epochs,
                                                                        n_samples, input_dim, latent_dim, batch_size,beta,
                                                                        gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                        model_path= model_path)


    plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time,n_rows_train, show_val= show_val, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,type = 'TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    print("-----------------------------------------------")

    print("VAE WEAK GENERATOR-------------------")
    time = datetime.now().strftime("%H-%M")
    model_name ="LSTM_VAE_WEAKGEN"
    model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_{model_name}_LD{latent_dim}_Beta{beta}_NT{n_rows_train}_{time}.keras'
    train_losses, val_losses, real_epochs, time, show_val, model_path, vae_weakGen = train_model(vae_weakGen,optimizer_weakGen,discriminator_optimizer, epochs,
                                                                        n_samples, input_dim, latent_dim, batch_size,beta,
                                                                        gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                        model_path= model_path)
    plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time,n_rows_train, show_val= show_val, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,type = 'TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    print("-----------------------------------------------")

    print("VAE 2X----------------------")
    time = datetime.now().strftime("%H-%M")
    model_name ="LSTM_VAE_2x"
    model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_{model_name}_LD{latent_dim}_Beta{beta}_NT{n_rows_train}_{time}.keras'
    train_losses, val_losses, real_epochs, time, show_val, model_path, vae_2x = train_model(vae_2x,optimizer_2x,discriminator_optimizer, epochs,
                                                                        n_samples, input_dim, latent_dim, batch_size,beta,
                                                                        gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                        model_path= model_path)
    
    plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time,n_rows_train, show_val= show_val, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,type = 'TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    print("-----------------------------------------------")
    
    print("VAE 2X WEAK GENERATOR------------------")
    time = datetime.now().strftime("%H-%M")
    model_name ="LSTM_VAE_2x_WEAKGEN"
    model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/BEST_{model_name}_LD{latent_dim}_Beta{beta}_NT{n_rows_train}_{time}.keras'
    train_losses, val_losses, real_epochs, time, show_val, model_path, VAE_2x_weakGen = train_model(VAE_2x_weakGen,optimizer_2x_weakGen,discriminator_optimizer, epochs,
                                                                        n_samples, input_dim, latent_dim, batch_size,beta,
                                                                        gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                        model_path= model_path)
   
    plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time,n_rows_train, show_val= show_val, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,model_name,type='TSNE', save = True, AWS = AWS, s3 = s3, BUCKET = BUCKET)
    print("-----------------------------------------------")


**Iterative Training on Saved Model**

In [ ]:
# Regular HYPERPARAMETERS 
#input_dim = np.size(processeddataframe['features'][0])
input_dim = 42
epochs = 50
n_samples = 1
# Best 512 settigns: AdamW with LR=1e-05, WD=1e-06, Beta1=0.85, Beta2=0.98  
learning_rate = 1e-04
weight_decay = 1e-06
# FactorVAE
learning_rate_disc = 5e-5
# Annealing and Early stop
steps_anneal = epochs * len(train_dataset)  
alpha = 0.0  # Minimum learning rate as a fraction of the initial learning rate
validation_method = "PLOT" # None, B_VAE, TC, B_TCVAE, PLOT
patience = 5  

# IMPORTANT 
latent_dim= 30  
beta = 0 # 20
beta_tc = 1.008 #1.008 # keep in mind this is tuned based on that we get tc loss = -244..

gamma = 0  # TC weight (typically between 1 and 10) 6
n_critic = 0
##################

AD = False

time = datetime.now().strftime("%H-%M")
model_name ="BEST_VAE"
new_model_path = f'/Users/SCHUGD/Desktop/MasterThesis/Code/Models/Iter_BEST_{model_name}_{time}.keras'

cosine_decay_schedule = CosineDecay(
    1e-03, steps_anneal, alpha=alpha
)


vae = keras.models.load_model(best_model_path)

#optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay, beta_1=beta1, beta_2=beta2)
#optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=weight_decay)
optimizer = vae.optimizer
discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate_disc)

train_losses, val_losses, real_epochs, time, show_val, model_path, vae = train_model(vae,optimizer,discriminator_optimizer, epochs,
                                                                    n_samples, input_dim, latent_dim, batch_size,beta,
                                                                      gamma, n_critic, steps_anneal, patience, time,beta_tc,validation_method,
                                                                      model_path = new_model_path)


plot_loss_curve(train_losses, val_losses , real_epochs, latent_dim, time, show_val= show_val)
#analyze_latent_variance(vae,train_dataset, test_dataset)
#analyze_kl_divergence(vae, train_dataset, test_dataset)
#get_latent_representations_label(vae, test_dataset,latent_dim, beta,n_critic,gamma,time, 'PCA', save = False)
get_latent_representations_label(vae, test_dataset, latent_dim, beta,n_critic,gamma,time,'TSNE', save = True)
#get_latent_representations_label(vae, train_dataset, 'TSNE')

if AD:
  reconstruction_AD = False
  latent_AD = True
  reconstruction_threshold, latent_threshold, mean_train, variance_train = get_threshold_from_train(model_path, reconstruction_AD, latent_AD)
  results, distances = anomaly_detection(vae, reconstruction_AD, latent_AD, mean_train, variance_train)
  reconstruction_accuracy , latent_accuracy = get_anomaly_detection_accuracy(reconstruction_AD, latent_AD, results,reconstruction_threshold,distances,latent_threshold,model_name, latent_dim,epochs,time)